# Route Simulation

## Route Network

In [191]:
from typing import List, Dict, Set

class Edge(object):
    def __init__(self, length:float, next_stop:Stop):
        self._length = length # Km
        self._next_stop = next_stop
    
    @property
    def length(self) -> int:
        return self._length
    
    @property
    def next_stop(self) -> Stop:
        return self._next_stop
    
    def __str__(self) -> str:
        return f"{self.length} Km to {self.next_stop}"
    
    def __repr__(self) -> str:
        return f"{self.length} Km to {self.next_stop}"

class Stop(object):
    def __init__(self, name:str, edges:Dict[str,Edge]=None, num_chargers:int=0, charger_rating:float=450):
        self._name = name
        self.edges = edges
        self.charger_rating = charger_rating # KWh
        self.charger_queues = [[] for i in range(num_chargers)]
        
    def shortest_charger_queue(self) -> int:
        shortest = -1
        for i in range(len(self.charger_queues)):
            if shortest == -1 or len(self.charger_queues[i]) < len(self.charger_queues[shortest]):
                shortest = i
        return shortest
    
    def buses_in_queue(self, i:int) -> int:
        return len(self.charger_queues[i])
    
    def add_bus_to_charger_queue(self, i:int, bus:Bus):
        self.charger_queues[i].append(bus)
        bus.is_charging = True
    
    def charge_all_buses(self, timestep:float): # timestamp in seconds
        for charger_queue in self.charger_queues:
            if len(charger_queue) >= 1:
                bus = charger_queue[0] # bus at the front of queue
                bus.battery_charge = min(bus.battery_charge + self.charger_rating * timestep / 3600, bus.battery_capacity) # KWh
                if bus.can_reach_next_stop(): # bus charged enough
                    charger_queue.pop(0) # Remove bus from queue
                    bus.is_charging = False
        
    def __eq__(self, other) -> bool:
        if not isinstance(other, Stop):
            return False
        return self.name == other.name and self.has_charger == other.has_charger
    
    def __hash__(self):
        return hash((self.name, self.has_charger))
    
    @property
    def name(self) -> str:
        return self._name
    
    def __str__(self) -> str:
        return f"{self.name}"
    
    def __repr__(self) -> str:
        return f"{self.name}"
    
class Route(object):
    def __init__(self, name:str):
        self._name = name
        self.stops = dict()
        self.buses = set()
        
    @property
    def name(self) -> str:
        return self._name
    
    def get_other_direction(self, cur_direction) -> str:
        for direction in stops:
            if direction != cur_direction:
                return direction
        return "No other direction"
    
    def add_stop(self, stop:Stop, direction:str):
        if direction not in self.stops:
            self.stops[direction] = list()
        self.stops[direction].append(stop)
        
    def add_bus(self, bus:Bus):
        self.buses.add(bus)
    
    def __str__(self) -> str:
        return f"{self.stops}"
    
    def __repr__(self) -> str:
        return f"{self.stops}"
            
class StopNetwork(object):
    def __init__(self):
        self.stops = dict() # Dictionary of Stop name -> Stop object
        self.routes = dict() # Dictionary of Route name -> Route object
        
    def add_edge(self, origin_name:str, dest_name:str, route_name:str, route_direction:str, length:float, origin_num_chargers:int=0, origin_charger_rating:float=450, dest_num_chargers:int=0, dest_charger_rating:float=450):
        # Populate stops dictionary
        if origin_name not in self.stops:
            self.stops[origin_name] = Stop(origin_name, {}, num_chargers=origin_num_chargers, charger_rating=origin_charger_rating)
        if dest_name not in self.stops:
            self.stops[dest_name] = Stop(dest_name, {}, num_chargers=dest_num_chargers, charger_rating=dest_charger_rating)
        self.stops[origin_name].edges[route_direction] = Edge(length, self.stops[dest_name])
        
        # Populate routes dictionary
        if route_name not in self.routes:
            self.routes[route_name] = Route(route_name)
        self.routes[route_name].add_stop(self.stops[origin_name], route_direction)
        
    def add_bus(self, bus_id:int, speed:float, cur_stop_name:str, route_name:str, route_direction:str, battery_capacity:float, battery_charge:float, energy_use_per_km:float):
        new_bus = Bus(bus_id, speed, self.stops[cur_stop_name], self.routes[route_name], route_direction, battery_capacity, battery_charge, energy_use_per_km)
        self.routes[route_name].add_bus(new_bus)
        
    def move_all_buses(self, timestep:float): # timestep in seconds
        for route in self.routes.values():
            for bus in route.buses:
                print(bus, bus.can_reach_next_stop(), bus.SOC())
                bus.move(timestep)
                
    def charge_all_buses(self, timestamp:float): # timestamp in seconds
        for stop in self.stops.values():
            stop.charge_all_buses(timestamp)

## Bus model

In [192]:
class Bus(object):
    def __init__(self, bus_id:int, speed:float, cur_stop:Stop, route:Route, route_direction:str, battery_capacity:float, battery_charge:float, energy_use_per_km:float, is_charging:bool=False):
        self._id = bus_id
        self.speed = speed # Km/h
        self.route = route
        self.cur_stop = cur_stop
        self.route_direction = route_direction
        self.distance_to_next_stop = cur_stop.edges[route_direction].length # Km
        self._battery_capacity = battery_capacity # KWh
        self.is_charging = is_charging
        self.battery_charge = battery_charge #KWh
        self.energy_use_per_km = energy_use_per_km # KWh / Km
        
    def SOC(self) -> float:
        return self.battery_charge / self.battery_capacity
    
    def can_reach_next_stop(self) -> bool:
        return self.battery_charge >= self.distance_to_next_stop * self.energy_use_per_km
    
    def move(self, timestep:float): # timestep in seconds
        if not self.is_charging: # Bus not charging
            distance_traveled = timestep * self.speed / 3600
            self.distance_to_next_stop -= distance_traveled
            self.battery_charge  = max(self.battery_charge - distance_traveled * self.energy_use_per_km, 0) # floor at 0
            if self.distance_to_next_stop <= 0: # arrived at next stop
                self.cur_stop = self.cur_stop.edges[self.route_direction].next_stop # Update stop
                if self.route_direction not in self.cur_stop.edges: # Change direction if end was reached
                    self.route_direction = self.route.get_other_direction(self.route_direction)
                self.distance_to_next_stop = self.cur_stop.edges[self.route_direction].length # Update distance to next stop

                shortest_charger_queue_index = self.cur_stop.shortest_charger_queue()
                if shortest_charger_queue_index != -1: # The stop has a charger
                    queue_size = self.cur_stop.buses_in_queue(shortest_charger_queue_index)
                    if queue_size <= 1 or not self.can_reach_next_stop():
                        self.cur_stop.add_bus_to_charger_queue(shortest_charger_queue_index, self)

    @property
    def id(self) -> int:
        return self._id
    
    @property
    def battery_capacity(self) -> int:
        return self._battery_capacity
    
    def __eq__(self, other) -> bool:
        if not isinstance(other, Bus):
            return False
        return self.id == other.id
    
    def __hash__(self):
        return hash(self.id)
    
    def __str__(self) -> str:
        return f"{self.id} {self.distance_to_next_stop} Km from {self.cur_stop}"
    
    def __repr__(self) -> str:
        return f"{self.id} {self.distance_to_next_stop} Km from {self.cur_stop}"

## Initialize stop network

In [204]:
stop_network = StopNetwork()

## M14D-SBS

In [205]:
direction = "M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST"
stop_names = [
    "DELANCEY ST/COLUMBIA ST",
    "COLUMBIA ST/RIVINGTON ST",
    "AV D/E HOUSTON ST",
    "AV D/E 5 ST",
    "E 10 ST/AV D",
    "AV C/E 11 ST",
    "E 14 ST/AV C",
    "E 14 ST/AV B",
    "E 14 ST/AV A",
    "E 14 ST/1 AV",
    "E 14 ST/2 AV",
    "E 14 ST/3 AV",
    "E 14 ST/4 AV",
    "E 14 ST/UNION SQ W",
    "W 14 ST/5 AV",
    "W 14 ST/6 AV",
    "W 14 ST/7 AV",
    "W 14 ST/8 AV",
    "W 14 ST/9 AV",
    "W 14 St/10 AV",
    "11 AV/W 15 ST",
    "11 AV / W 17 ST"
]
distances = [
    0.167,
    0.280,
    0.255,
    0.387,
    0.244,
    0.318,
    0.175,
    0.226,
    0.550,
    0.339,
    0.199,
    0.178,
    0.265,
    0.195,
    0.309,
    0.259,
    0.278,
    0.286,
    0.140,
    0.268,
    0.126
]
for i in range(1, len(stop_names)):
    origin_name = stop_names[i - 1]
    dest_name = stop_names[i]
    length = distances[i - 1]
    stop_network.add_edge(origin_name, dest_name, "M14D-SBS", direction, length, 2, 450, 2)

In [206]:
bus_id = 4950
speed = 10.33 # Km/h
cur_stop_name = "DELANCEY ST/COLUMBIA ST"
route_name = "M14D-SBS"
route_direction = "M14D-SBS to SELECT BUS CHLSEA PIERS 11 AV via 14 ST"
battery_capacity = 140 # KWh
battery_charge = 140 # KWh
energy_use_per_km = 0.586 # KWh / Km
stop_network.add_bus(bus_id, speed, cur_stop_name, route_name, route_direction, battery_capacity, battery_charge, energy_use_per_km)

In [207]:
while True:
    try:
        stop_network.move_all_buses(1)
        stop_network.charge_all_buses(1)
    except:
        break

4950 0.167 Km from DELANCEY ST/COLUMBIA ST True 1.0
4950 0.16413055555555556 Km from DELANCEY ST/COLUMBIA ST True 0.9999879893253968
4950 0.1612611111111111 Km from DELANCEY ST/COLUMBIA ST True 0.9999759786507935
4950 0.15839166666666665 Km from DELANCEY ST/COLUMBIA ST True 0.9999639679761902
4950 0.1555222222222222 Km from DELANCEY ST/COLUMBIA ST True 0.999951957301587
4950 0.15265277777777775 Km from DELANCEY ST/COLUMBIA ST True 0.9999399466269837
4950 0.1497833333333333 Km from DELANCEY ST/COLUMBIA ST True 0.9999279359523805
4950 0.14691388888888884 Km from DELANCEY ST/COLUMBIA ST True 0.9999159252777773
4950 0.1440444444444444 Km from DELANCEY ST/COLUMBIA ST True 0.999903914603174
4950 0.14117499999999994 Km from DELANCEY ST/COLUMBIA ST True 0.9998919039285707
4950 0.1383055555555555 Km from DELANCEY ST/COLUMBIA ST True 0.9998798932539675
4950 0.13543611111111103 Km from DELANCEY ST/COLUMBIA ST True 0.9998678825793643
4950 0.13256666666666658 Km from DELANCEY ST/COLUMBIA ST True 0.

4950 0.13187222222222206 Km from E 14 ST/UNION SQ W True 0.9960852840475294
4950 0.1290027777777776 Km from E 14 ST/UNION SQ W True 0.9960732733729261
4950 0.12613333333333315 Km from E 14 ST/UNION SQ W True 0.9960612626983228
4950 0.12326388888888871 Km from E 14 ST/UNION SQ W True 0.9960492520237196
4950 0.12039444444444428 Km from E 14 ST/UNION SQ W True 0.9960372413491163
4950 0.11752499999999984 Km from E 14 ST/UNION SQ W True 0.9960252306745131
4950 0.1146555555555554 Km from E 14 ST/UNION SQ W True 0.9960132199999099
4950 0.11178611111111096 Km from E 14 ST/UNION SQ W True 0.9960012093253066
4950 0.10891666666666652 Km from E 14 ST/UNION SQ W True 0.9959891986507033
4950 0.10604722222222208 Km from E 14 ST/UNION SQ W True 0.9959771879761001
4950 0.10317777777777765 Km from E 14 ST/UNION SQ W True 0.9959651773014968
4950 0.10030833333333321 Km from E 14 ST/UNION SQ W True 0.9959531666268936
4950 0.09743888888888877 Km from E 14 ST/UNION SQ W True 0.9959411559522904
4950 0.0945694